# Batch scheduling of a fermentation process

Industrial fermentation processes are generally batch operations. The process consists of a
number of phases or stages, each of which requires a considerable amount of time. Because
of the time factor, the system cannot react quickly to variations in demand, so that the
scheduling of production in each stage is of considerable importance.

Consider a fermentation process which has five stages: (1) Mixing and cooking [$M$,$m$]; (2) Fermentation [$F$,$f$]; (3) Purification [$P$,$p$]; (4) Blending and packaging [$B$,$b$]; (5) Warehousing [$W$,$w$].

$m_i$ ($f_i$, $p_i$, etc.) is the quantity that goes into stage $m$ (from the previous stage) in day $i$. $d_i$ is the demand (by customers) from the warehouse in period $i$. The demand $d_i$ is known (but not precisely). Given $d_i$, a set of values $m_i$, $f_i$, $p_i$, $b_i$, $w_i$ constitutes a production schedule. The capital letters indicate the total quantity in the stage, i.e. $M_j$ is the total amount of material in the mixing stage in period $j$.

### A note on unit selection
It is a good idea to select the unit you will be using from the start, especially considering factors of 10 (such as kg, tonnes etc.). In this example, we will always use $1000L$ as our unit.

## Initialization and variable selection
As you may expect, we have to introduce variables for each step of the process, both for the lower case value (ingoing) as well as the upper case value (amount contained).

> Note that due to 0-indexing, we have $0\leq i\leq N-1$.

In [44]:
import xpress as xp
import numpy as np
import matplotlib.pyplot as plt
import math

model = xp.problem("Batch scheduling")

# This is required due to 1 indexing instead of 0 indexing
n = 105
nRange = range(n)
t_m = 3
A_max = 8
Z_b = 5
t_min_fermentation = 10
t_max_fermentation = 30
t_min_purification = 2
t_max_purification = 5
C_m = 350; mu = 2;
C_f = 410; phi = 5;
C_p = 380; pi = 7;
C_b = 250; beta = 2;
C_w = 200; chi = 1;

# This is the demand. We only need it later but it is good practice to define it beforehand.
d = np.array([0 for i in range(20)])
#d = np.append(d,np.array([3 for i in range(N-20)]))
#d = np.append(d,np.array([4 for i in range(N-20)]))
#d = np.append(d,np.array([4.5 for i in range(N-20)]))
d = np.append(d, np.array([5+math.sin(i/10) for i in range(N-20)]))

In [ ]:
m = {}
M = {}

f = {}
F = {}

p = {}
P = {}

b = {}
B = {}

w = {}
W = {}

model.addVariable(m,M,f,F,p,P,b,B,w,W)

### Linking variables together
It is often a good idea to get the "logical" constraints out of the way first, i.e. the constraints that link variable $A$ and $B$. In this case, we link the "inflow" variables with the overall quantities.

For example, let's consider the fermentation process. Mathematically, this can be described as:

$F_j = \sum \limits_{i=0}^{j} (f_{i} - p_{i})$,

where $F_j$ is the total quanitity in the fermentation at period $j$.

In [ ]:
mixing_balance = ()
fermentation_balance = ()
purification_balance = ()
blending_balance = ()
warehousing_balance = ()

model.addConstraint(mixing_balance, fermentation_balance, purification_balance, blending_balance, warehousing_balance)